In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
data_dict = pd.read_table("2025_Problem_C_Data\\data_dictionary.csv", encoding="Windows-1252",sep=",")

# 变量说明

In [ ]:
data_dict.iloc[0:8,]

In [ ]:
data_dict.iloc[11:14,]

In [ ]:
data_dict.iloc[17:36,]

In [ ]:
data_dict.iloc[39:50,]

以下是对该文件内容的翻译：

---

### **文件 1：夏季奥运会奖牌统计 (summerOly_medal_counts.csv)**

| 变量名称   | 解释            | 示例           |
|--------|---------------|--------------|
| Rank   | 国家奖牌总数排名      | "1, 2"       |
| NOC    | 该奥运会记录的国家名称   | "中国, 法国"     |
| Gold   | 国家获得的金牌数量     | "0, 1, 2"    |
| Silver | 国家获得的银牌数量     | "0, 1, 2"    |
| Bronze | 国家获得的铜牌数量     | "0, 1, 2"    |
| Total  | 国家获得的奖牌总数     | "0, 1, 2"  |
| Year   | 奥运会年份         | "1896, 1900" |

---

### **文件 2：夏季奥运会举办地 (summerOly_hosts.csv)**

| 变量名称 | 解释                                       | 示例                      |
|----------|--------------------------------------------|---------------------------|
| Year     | 奥运会年份                                 | "1896, 1900"              |
| Host     | 奥运会举办地（城市，国家）                 | "圣路易斯, 美国"          |

---

### **文件 3：夏季奥运会项目 (summerOly_programs.csv)**

| 变量名称       | 解释              | 示例          |
|------------|-----------------|-------------|
| Sport      | 体育类别            | "水上运动, 自行车" |
| Discipline | 体育类别内的具体项目      | "艺术体操, 跳水"  |
| Code       | 项目的代码           | "SWA, DIV"  |
| Year       | 该项目在某年奥运会中的比赛数量 | "0, 1"      |

#### 数据说明：
- **体育和项目 (Sport and Discipline)**：包括当前和已取消的夏季奥运会官方项目，按国际奥委会（IOC）使用的名称按字母顺序排列。
- **代码 (Code)**：每个项目由 IOC 分配一个独特的 3 字符标识符代码。
- **体育管理机构**：列出与奥运会体育相关的国际管理机构。
- **年份 (1896-2028)**：列的四位数代表相关奥运会的年份。
- 表格中的数字表示该项目在该年举办的比赛数量；某些项目可能标记为展示项目（非正式项目）。

---

### **文件 4：夏季奥运会运动员数据 (summerOly_athletes.csv)**

| 变量名称 | 解释                                   | 示例                       |
|----------|----------------------------------------|----------------------------|
| Name     | 运动员姓名                             | "蔡赟, 卡尔·刘易斯"        |
| Sex      | 运动员性别                             | "女 (F), 男 (M)"           |
| Team     | 国家名称（奥运会记录的）               | "中国, 法国"               |
| NOC      | 国家代码                               | "CHN, FRA"                 |
| Year     | 奥运会年份                             | "1896, 1900"               |
| Season   | 奥运会类型                             | 夏季 (Summer)              |
| City     | 奥运会举办城市                         | "北京, 巴黎"               |
| Sport    | 体育类别                               | "水上运动, 自行车"         |
| Event    | 项目名称                               | "体操男子吊环"             |
| Medal    | 获得的奖牌类型                         | "无奖牌, 金牌 (Gold)"      |

---

### **备注**
1. **1906年奥运会**：原计划每四年举办一次，地位与正式奥运会相同，但仅在 1906 年于雅典举办了一次，之后取消。
2. **冬季项目迁移**：在 1924 年之前，一些冰上项目（如花样滑冰和冰球）在夏季奥运会中举办，1924 年起转移至冬奥会。
3. **管理机构变化**：如拳击的管理机构从 2023 年起由 AIBA 改为 IBA。

---

# 获取分析历史奖牌数据

In [ ]:
history_medal_counts = pd.read_csv("2025_Problem_C_Data\\summerOly_medal_counts.csv", encoding="Windows-1252",sep=",")

In [ ]:
history_medal_counts

# 对于历史上已经消失或者合并的国家的数据进行清洗

In [ ]:
# 获取国家的名称
noc_names = history_medal_counts["NOC"].unique()
print(noc_names)

In [ ]:
# Step 1: Redefine the removal and merge rules based on updated requirements
# Countries/regions to remove entirely
final_remove_nocs = [
    "Bohemia", "Australasia", "Russian Empire", "Czechoslovakia",
    "Unified Team", "Mixed team", "East Germany", "West Germany",
    "United Team of Germany", "FR Yugoslavia", "Serbia and Montenegro",
    "Soviet Union"
]

# Update merge mappings
final_merge_nocs = {
    "Formosa": "Chinese Taipei"  # Merge Formosa with Chinese Taipei
    # Note: China, Hong Kong, and Macau remain separate
}

# Step 2: Start cleaning process from original data
final_cleaned_data = history_medal_counts[~history_medal_counts["NOC"].isin(final_remove_nocs)]

# Replace NOC names based on updated merge mapping
final_cleaned_data["NOC"] = final_cleaned_data["NOC"].replace(final_merge_nocs)

# Step 3: Recalculate totals and ranks with the updated cleaning rules
final_cleaned_data = (
    final_cleaned_data.groupby(["Year", "NOC"], as_index=False)
    .agg({"Gold": "sum", "Silver": "sum", "Bronze": "sum"})
)
final_cleaned_data["Total"] = final_cleaned_data["Gold"] + final_cleaned_data["Silver"] + final_cleaned_data["Bronze"]

# Rank NOCs within each year
final_cleaned_data["Rank"] = (
    final_cleaned_data.sort_values(["Year", "Total", "Gold", "Silver"], ascending=[True, False, False, False])
    .groupby("Year")
    .cumcount() + 1
)

# 保存数据
final_cleaned_data.head()
cleaned_medal_data = final_cleaned_data
cleaned_medal_data.to_csv("2025_Problem_C_Data_cleaned\\cleaned_medal_data.csv", index=False)


In [ ]:
# # 遍历分组
# for name, group in final_cleaned_data.groupby("NOC"):
#     # 绘制每个 NOC 的奖牌柱状图
#     group.plot(x="Year", y=["Gold", "Silver", "Bronze"], kind="bar", title=name)
#     plt.tight_layout()  # 调整布局
#     plt.show()  # 显示每个图


- 清洗后得到的各个国家和地区的历史奖牌的柱状图如图

# 读取项目表

In [ ]:
program_data = pd.read_csv("2025_Problem_C_Data\\summerOly_programs.csv", encoding="latin1",sep=",")

In [ ]:
program_data.info()

In [ ]:
print(pd.unique(program_data["Discipline"]))

In [ ]:
# 清除不太可能存在于2028奥运会当中的项目
unlikely_events = [
    "Boxing", "Weightlifting", "Breaking",
    "Croquet", "Softball", "Lacrosse", 
    "Squash", "Cricket","Water Motorsports","Skating","Ice Hockey"
]

# Remove these unlikely events from the dataset
filtered_program_data = program_data[~program_data['Sport'].isin(unlikely_events)]
filtered_program_data.info()



1. **拳击（Boxing）**：由于国际拳击联合会的管理问题，拳击项目未被列入2028年洛杉矶奥运会的初始项目名单。尽管如此，如果相关问题得到解决，拳击仍有可能被重新纳入。 ([澎湃新闻](https://www.thepaper.cn/newsDetail_forward_15781574?utm_source=chatgpt.com))

2. **举重（Weightlifting）**：因反兴奋剂方面的历史问题，举重项目也未被列入初始名单。国际举重联合会需要进行改革，以确保该项目能够回归奥运会。 ([澎湃新闻](https://www.thepaper.cn/newsDetail_forward_15781574?utm_source=chatgpt.com))


1. **霹雳舞（Breaking）**：尽管在2024年巴黎奥运会上首次亮相，但霹雳舞未被纳入2028年洛杉矶奥运会的项目。 ([Diario AS](https://as.com/juegos_olimpicos/proxima-parada-hollywood-7000-millones-sin-coches-criquet-n/?utm_source=chatgpt.com))

2. **槌球（Croquet）**、**壘球（Softball）**、**长曲棍球（Lacrosse）**、**壁球（Squash）**、**板球（Cricket）**：这些项目在历史上曾出现在奥运会上，但在近期的奥运会中并未包含。然而，洛杉矶奥组委提议将棒垒球、板球、腰旗橄榄球、棍网球和壁球作为新增大项纳入2028年奥运会。 


In [ ]:
# 填充缺失值
# 仅有Modern Pentathlon项目不存在归属，直接使用其名字作为discipline
filtered_program_data.fillna("Modern Pentathlon", inplace=True)

In [ ]:
# 转换类型
# 遍历所有列，将日期列的数据类型转换为整数
for col in filtered_program_data.columns:
    if col not in ["Discipline", "Sport", "Sports Governing Body","Code"]:
        # 将列数据转换为整数，跳过可能导致错误的数据
        filtered_program_data[col] = pd.to_numeric(filtered_program_data[col], errors='coerce').fillna(0).astype(int)

In [ ]:
filtered_program_data.info()

In [ ]:
filtered_program_data.head(10)

In [ ]:
# 保存清洗后的数据
filtered_program_data.to_csv("2025_Problem_C_Data_cleaned\\filtered_program_data.csv", encoding="utf-8",index=False)


In [ ]:
# 只保留1990年以来的奥运会的项目情况
# 删除第 4 到第 25 列
filtered_program_data.drop(columns=filtered_program_data.columns[4:26], inplace=True)
filtered_program_data.info()

In [ ]:
rows_with_na = filtered_program_data[filtered_program_data.isna().any(axis=1)]
print("含缺失值的行：")
print(rows_with_na)

# 读取东道主的数据

In [ ]:
host_data = pd.read_csv("2025_Problem_C_Data\\summerOly_hosts.csv")

In [ ]:
host_data

# 读取并清洗运动员的数据

In [ ]:
athletics_data = pd.read_csv("2025_Problem_C_Data\\summerOly_athletes.csv")

In [ ]:
athletics_data

In [ ]:
athletics_data.isna().sum()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Step 1: 加载数据
athletes = pd.read_csv('2025_Problem_C_Data/summerOly_athletes.csv', encoding='latin1')
hosts = pd.read_csv('2025_Problem_C_Data/summerOly_hosts.csv', encoding='latin1')


# Step 2: 清洗数据
# 移除诡异的字符
hosts['Host'] = hosts['Host'].str.replace(r'[Â\xa0]', '', regex=True).str.strip()

In [ ]:

# 移除无奖牌的数据
athletes = athletes[athletes['Medal'] != 'No medal']

# Group data by Team, Year, Sport, and Medal, then count the occurrences
grouped_data = athletes.groupby(['Team', 'Year', 'Sport', 'Medal']).size().reset_index(name='Count')

# Pivot the data to get Medal types as columns
pivot_table = grouped_data.pivot_table(
    index=['Team', 'Year', 'Sport'],
    columns='Medal',
    values='Count',
    fill_value=0
).reset_index()


In [ ]:
pivot_table.info()

In [ ]:
print(pivot_table.head(4))

In [ ]:
# 删除1990前的数据
pivot_table = pivot_table[pivot_table["Year"]>1990]

In [ ]:
pivot_table.info()

In [ ]:
# 标准化国家名称
pivot_table['Team'] = pivot_table['Team'].str.replace(r'-\d+$', '', regex=True)


In [ ]:
# 保存表格
output_path = '2025_Problem_C_Data_cleaned/olympic_medal_analysis.csv'
pivot_table.to_csv(output_path, index=False)

# 合并表格数据,将cleaned_medal_data当中的数据与运动项目的数据进行合并，同时考虑东道主效应的影响

In [ ]:
import pandas as pd

# 加载数据文件
file_path_original = '2025_Problem_C_Data_cleaned/olympic_medal_analysis.csv'
file_path_new = '2025_Problem_C_Data_cleaned/cleaned_medal_data.csv'

data = pd.read_csv(file_path_original)
new_data = pd.read_csv(file_path_new)
data = pd.read_csv(file_path_original)
new_data = pd.read_csv(file_path_new)

# 手动创建 1992 年以来的东道主国家清洁数据
year_host_mapping = {
    1992: "Spain",
    1996: "United States",
    2000: "Australia",
    2004: "Greece",
    2008: "China",
    2012: "Great Britain",
    2016: "Brazil",
    2020: "Japan",
    2024: "France"
}
# 标准化国家名称，移除例如 '-1' 的后缀
data['Team'] = data['Team'].str.replace(r'-\d+$', '', regex=True)
data.rename(columns={'Team': 'NOC'}, inplace=True)

# 添加东道主效应
# 标记每个国家是否是该年的东道主
data['Is_Host'] = data.apply(lambda row: 1 if row['Year'] in year_host_mapping and row['NOC'] == year_host_mapping[row['Year']] else 0, axis=1)
# 步骤 1: 数据清洗
# 确保存在 'Total' 列，如果没有则通过求和 'Gold', 'Silver', 'Bronze' 列创建
if 'Total' not in data.columns:
    data['Total'] = data[['Gold', 'Silver', 'Bronze']].sum(axis=1)

# 步骤 2: 创建按国家、年份和运动项目统计的奖牌数据透视表
# 创建金牌数据表
gold_pivot = data.pivot_table(
    index=['NOC', 'Year','Is_Host'], 
    columns='Sport', 
    values='Gold', 
    aggfunc='sum', 
    fill_value=0
)
gold_pivot = gold_pivot.drop(columns=unlikely_events, errors='ignore')  # 移除不可能的项目
gold_summary = data.groupby(['NOC', 'Year','Is_Host'])[['Gold']].sum()
gold_data = pd.concat([gold_summary, gold_pivot], axis=1).reset_index()
gold_file_path = '2025_Problem_C_Data_cleaned/gold_medal_data.csv'
gold_data.to_csv(gold_file_path, index=False)

# 创建银牌数据表
silver_pivot = data.pivot_table(
    index=['NOC', 'Year','Is_Host'], 
    columns='Sport', 
    values='Silver', 
    aggfunc='sum', 
    fill_value=0
)
silver_pivot = silver_pivot.drop(columns=unlikely_events, errors='ignore')
silver_summary = data.groupby(['NOC','Year','Is_Host'])[['Silver']].sum()
silver_data = pd.concat([silver_summary, silver_pivot], axis=1).reset_index()
silver_file_path = '2025_Problem_C_Data_cleaned/silver_medal_data.csv'
silver_data.to_csv(silver_file_path, index=False)

# 创建铜牌数据表
bronze_pivot = data.pivot_table(
    index=['NOC', 'Year','Is_Host'], 
    columns='Sport', 
    values='Bronze', 
    aggfunc='sum', 
    fill_value=0
)

bronze_pivot = bronze_pivot.drop(columns=unlikely_events, errors='ignore')
bronze_summary = data.groupby(['NOC', 'Year','Is_Host'])[['Bronze']].sum()
bronze_data = pd.concat([bronze_summary, bronze_pivot], axis=1).reset_index()
bronze_file_path = '2025_Problem_C_Data_cleaned/bronze_medal_data.csv'
bronze_data.to_csv(bronze_file_path, index=False)

# 输出保存的文件路径
print("Gold medals saved to:", gold_file_path)
print("Silver medals saved to:", silver_file_path)
print("Bronze medals saved to:", bronze_file_path)

# 建立模型

In [ ]:

# 加载数据
file_path_gold = '2025_Problem_C_Data_cleaned/gold_medal_data.csv'
file_path_silver = '2025_Problem_C_Data_cleaned/silver_medal_data.csv'
file_path_bronze = '2025_Problem_C_Data_cleaned/bronze_medal_data.csv'

gold_data = pd.read_csv(file_path_gold)
silver_data = pd.read_csv(file_path_silver)
bronze_data = pd.read_csv(file_path_bronze)

# 检查数据结构
gold_data.head()

In [ ]:
silver_data.head()

In [ ]:
bronze_data.head()

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import GridSearchCV
# 
# # 通用函数：补全年份数据
# def optimized_complete_years(data, start_year=1992, end_year=2024):
#     """
#     Optimized version of complete_years function to fill missing years for each country.
#     
#     Parameters:
#         data (pd.DataFrame): Input data with columns 'NOC', 'Year', and other metrics.
#         start_year (int): Starting year for the completion.
#         end_year (int): Ending year for the completion.
# 
#     Returns:
#         pd.DataFrame: Data with missing years filled for each country.
#     """
#     # Generate a complete multi-index for all countries and years
#     all_years = range(start_year, end_year + 1)
#     all_countries = data['NOC'].unique()
#     multi_index = pd.MultiIndex.from_product([all_countries, all_years], names=['NOC', 'Year'])
# 
#     # Reindex the data using the complete multi-index
#     data.set_index(['NOC', 'Year'], inplace=True)
#     complete_data = data.reindex(multi_index, fill_value=0).reset_index()
# 
#     # Ensure Is_Host is appropriately handled (default to 0 for missing years)
#     if 'Is_Host' in complete_data.columns:
#         complete_data['Is_Host'] = complete_data['Is_Host'].fillna(0).astype(int)
#     else:
#         complete_data['Is_Host'] = 0
# 
#     return complete_data
# 
# # 通用函数：特征设计
# def create_features_and_targets(data, target_column):
#     features = []
#     targets = []
#     
#     if 'NOC' not in data.columns:
#         raise KeyError("The input data must contain a 'NOC' column.")
#     
#     for noc in data['NOC'].unique():
#         noc_data = data[data['NOC'] == noc]
# 
#         for i, row in noc_data.iterrows():
#             if row['Year'] < 2028:
#                 # 构建基础特征
#                 feature_row = {
#                     'Year': row['Year'],
#                     'Is_Host': row['Is_Host'],
#                     'NOC': row['NOC'],  # 保留NOC列
#                 }
# 
#                 # 历史项目表现（考虑历史数据的平滑处理）
#                 for col in data.columns:
#                     if col not in ['NOC', 'Year', 'Is_Host', target_column]:
#                         past_data = noc_data.loc[noc_data['Year'] < row['Year'], col]
#                         
#                         # 使用中位数替代均值，防止异常值的影响
#                         feature_row[f'{col}_median'] = past_data.median()
#                         feature_row[f'{col}_std'] = past_data.std()
#                         
#                         # 使用指数平滑来捕捉趋势
#                         smoothed_data = past_data.ewm(span=3).mean()  # 5年为平滑窗口
#                         feature_row[f'{col}_trend'] = smoothed_data.diff().mean()
# 
#                 features.append(feature_row)
#                 targets.append(row[target_column])
# 
#     features_df = pd.DataFrame(features)
#     features_df.fillna(0, inplace=True)
# 
#     return features_df, np.array(targets)
# 
# # 滚动窗口验证函数
# def rolling_window_validation(data, target_column, n_years=5):
#     unique_years = sorted(data['Year'].unique())
#     results = []
#     mse_list = []
#     r2_list = []
# 
#     for i in range(n_years, len(unique_years)):
#         train_years = unique_years[:i]
#         val_year = unique_years[i]
# 
#         # 划分训练集和验证集
#         train_data = data[data['Year'].isin(train_years)]
#         val_data = data[data['Year'] == val_year]
# 
#         X_train, y_train = create_features_and_targets(train_data, target_column)
#         X_val, y_val = create_features_and_targets(val_data, target_column)
# 
#         # 保留NOC列以便后续分析
#         noc_train = X_train['NOC']
#         noc_val = X_val['NOC']
# 
#         X_train = X_train.drop(columns=['Year', 'NOC'])
#         X_val = X_val.drop(columns=['Year', 'NOC'])
# 
#         # 训练模型
#         model = GradientBoostingRegressor(random_state=42)
#         model.fit(X_train, y_train)
# 
#         # 验证模型
#         y_pred = model.predict(X_val)
#         mse = mean_squared_error(y_val, y_pred)
#         r2 = r2_score(y_val, y_pred)
# 
#         results.append({
#             'Year': val_year,
#             'MSE': mse,
#             'R2': r2,
#             'NOCs': noc_val.tolist()  # 添加NOC信息
#         })
#         mse_list.append(mse)
#         r2_list.append(r2)
# 
#     # 可视化滚动窗口验证结果
#     plt.figure(figsize=(12, 6))
#     plt.plot(unique_years[n_years:], mse_list, label='MSE', marker='o')
#     plt.plot(unique_years[n_years:], r2_list, label='R2', marker='o')
#     plt.title(f'Rolling Window Validation for {target_column}')
#     plt.xlabel('Year')
#     plt.ylabel('Score')
#     plt.legend()
#     plt.grid(True)
#     plt.show()
#     plt.close()
# 
#     return pd.DataFrame(results)
# 
# # 通用函数：预测 2028 年表现
# def predict_2028_per_country(data, target_column):
#     countries = data['NOC'].unique()
#     events = [col for col in data.columns if col not in ['NOC', 'Year', 'Is_Host', target_column]]
#     all_results = []
# 
#     # 定义超参数网格
#     param_grid = {
#         'n_estimators': [50, 100, 200],  # 估计器数量
#         'max_depth': [3, 4, 5, 6],        # 树的最大深度
#         'learning_rate': [0.01, 0.05, 0.1, 0.2],  # 学习率
#         'subsample': [0.8, 0.9, 1.0]      # 子样本比例
#     }
# 
#     # 初始化超参数优化的模型
#     model = GradientBoostingRegressor(random_state=42)
# 
#     for country in countries:
#         # 提取该国家的数据
#         country_data = data[data['NOC'] == country]
# 
#         for event in events:
#             # 为每个项目单独训练和预测
#             event_data = country_data[['Year', 'Is_Host', target_column, event, 'NOC']].rename(columns={event: 'Event_Score'})
#             X, y = create_features_and_targets(event_data, target_column)
# 
#             # 检查数据是否足够训练模型
#             if len(X) < 2:
#                 print(f"Insufficient data for {country} in {event}. Skipping...")
#                 continue
# 
#             # 分离训练特征和标签
#             X_train = X[X['Year'] < 2028].drop(columns=['Year', 'NOC'])
#             y_train = y[:len(X_train)]
# 
#             # 使用 GridSearchCV 进行超参数优化
#             grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
#             grid_search.fit(X_train, y_train)
# 
#             # 输出最佳参数
#             print(f"Best parameters for {country} in {event}: {grid_search.best_params_}")
# 
#             # 使用最佳参数训练模型
#             best_model = grid_search.best_estimator_
# 
#             # 动态构建 2028 年特征
#             feature_2028 = {}
#             for col in X_train.columns:
#                 feature_2028[col] = X_train[col].mean()
# 
#             feature_2028['Is_Host'] = 1 if country == 'USA' else 0
#             feature_2028 = pd.DataFrame([feature_2028])
# 
#             # 预测 2028 年的表现
#             y_2028_pred = best_model.predict(feature_2028)
# 
#             # 构建预测结果
#             all_results.append({
#                 'NOC': country,
#                 'Event': event,
#                 'Year': 2028,
#                 'Predicted_Medals': max(0, int(np.floor(y_2028_pred[0])))  # 保证非负整数
#             })
# 
#     return pd.DataFrame(all_results)


In [ ]:
# # 使用修正后的代码进行预测
# bronze_data = optimized_complete_years(bronze_data)
# gold_data = optimized_complete_years(gold_data)
# silver_data = optimized_complete_years(silver_data)
# bronze_data.to_csv('bronze_data.csv', index=False)
# gold_data.to_csv('gold_data.csv', index=False)
# silver_data.to_csv('silver_data.csv', index=False)

In [ ]:

# # 预测
# gold_predictions = predict_2028_per_country(gold_data, 'Gold')
# silver_predictions = predict_2028_per_country(silver_data, 'Silver')
# bronze_predictions = predict_2028_per_country(bronze_data, 'Bronze')
# 
# # 保存预测结果
# gold_predictions.to_excel('gold_predictions_2028_debug.xlsx', index=False)
# silver_predictions.to_excel('silver_predictions_2028_debug.xlsx', index=False)
# bronze_predictions.to_excel('bronze_predictions_2028_debug.xlsx', index=False)


In [ ]:
# # 滚动窗口验证
# gold_validation_results = rolling_window_validation(gold_data, 'Gold')
# silver_validation_results = rolling_window_validation(silver_data, 'Silver')
# bronze_validation_results = rolling_window_validation(bronze_data, 'Bronze')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# 通用函数：补全年份数据
def optimized_complete_years(data, start_year=1992, end_year=2024):
    """
    Optimized version of complete_years function to fill missing years for each country.
    
    Parameters:
        data (pd.DataFrame): Input data with columns 'NOC', 'Year', and other metrics.
        start_year (int): Starting year for the completion.
        end_year (int): Ending year for the completion.

    Returns:
        pd.DataFrame: Data with missing years filled for each country.
    """
    # Generate a complete multi-index for all countries and years
    all_years = range(start_year, end_year + 1)
    all_countries = data['NOC'].unique()
    multi_index = pd.MultiIndex.from_product([all_countries, all_years], names=['NOC', 'Year'])

    # Reindex the data using the complete multi-index
    data.set_index(['NOC', 'Year'], inplace=True)
    complete_data = data.reindex(multi_index, fill_value=0).reset_index()

    # Ensure Is_Host is appropriately handled (default to 0 for missing years)
    if 'Is_Host' in complete_data.columns:
        complete_data['Is_Host'] = complete_data['Is_Host'].fillna(0).astype(int)
    else:
        complete_data['Is_Host'] = 0

    return complete_data

# 通用函数：特征设计
def create_features_and_targets(data, target_column):
    features = []
    targets = []
    
    if 'NOC' not in data.columns:
        raise KeyError("The input data must contain a 'NOC' column.")
    
    for noc in data['NOC'].unique():
        noc_data = data[data['NOC'] == noc]

        for i, row in noc_data.iterrows():
            if row['Year'] < 2028:
                # 构建基础特征
                feature_row = {
                    'Year': row['Year'],
                    'Is_Host': row['Is_Host'],
                    'NOC': row['NOC'],  # 保留NOC列
                }

                # 历史项目表现（考虑历史数据的平滑处理）
                for col in data.columns:
                    if col not in ['NOC', 'Year', 'Is_Host', target_column]:
                        past_data = noc_data.loc[noc_data['Year'] < row['Year'], col]
                        
                        # 使用中位数替代均值，防止异常值的影响
                        feature_row[f'{col}_median'] = past_data.median()
                        feature_row[f'{col}_std'] = past_data.std()
                        
                        # 使用指数平滑来捕捉趋势
                        smoothed_data = past_data.ewm(span=3).mean()  # 5年为平滑窗口
                        feature_row[f'{col}_trend'] = smoothed_data.diff().mean()

                features.append(feature_row)
                targets.append(row[target_column])

    features_df = pd.DataFrame(features)
    features_df.fillna(0, inplace=True)

    return features_df, np.array(targets)

# 滚动窗口验证函数
def rolling_window_validation(data, target_column, n_years=5):
    unique_years = sorted(data['Year'].unique())
    results = []
    mse_list = []
    r2_list = []

    for i in range(n_years, len(unique_years)):
        train_years = unique_years[:i]
        val_year = unique_years[i]

        # 划分训练集和验证集
        train_data = data[data['Year'].isin(train_years)]
        val_data = data[data['Year'] == val_year]

        X_train, y_train = create_features_and_targets(train_data, target_column)
        X_val, y_val = create_features_and_targets(val_data, target_column)

        # 保留NOC列以便后续分析
        noc_train = X_train['NOC']
        noc_val = X_val['NOC']

        X_train = X_train.drop(columns=['Year', 'NOC'])
        X_val = X_val.drop(columns=['Year', 'NOC'])

        # 训练模型
        model = GradientBoostingRegressor(random_state=42)
        model.fit(X_train, y_train)

        # 验证模型
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)

        results.append({
            'Year': val_year,
            'MSE': mse,
            'R2': r2,
            'NOCs': noc_val.tolist()  # 添加NOC信息
        })
        mse_list.append(mse)
        r2_list.append(r2)

    # 可视化滚动窗口验证结果
    plt.figure(figsize=(12, 6))
    plt.plot(unique_years[n_years:], mse_list, label='MSE', marker='o')
    plt.plot(unique_years[n_years:], r2_list, label='R2', marker='o')
    plt.title(f'Rolling Window Validation for {target_column}')
    plt.xlabel('Year')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.close()

    return pd.DataFrame(results)

# 通用函数：预测 2028 年表现
def predict_2028_per_country(data, target_column):
    countries = data['NOC'].unique()
    events = [col for col in data.columns if col not in ['NOC', 'Year', 'Is_Host', target_column]]
    all_results = []

    # 定义超参数网格
    param_grid = {
        'n_estimators': [50, 100, 150],  # 估计器数量
        'max_depth': [2, 3, 4, 5],        # 树的最大深度
        'learning_rate': [0.01, 0.05, 0.1, 0.2],  # 学习率
        'subsample': [0.7, 0.8, 1.0]      # 子样本比例
    }

    # 初始化超参数优化的模型
    model = GradientBoostingRegressor(random_state=42)

    for country in countries:
        # 提取该国家的数据
        country_data = data[data['NOC'] == country]

        for event in events:
            # 为每个项目单独训练和预测
            event_data = country_data[['Year', 'Is_Host', target_column, event, 'NOC']].rename(columns={event: 'Event_Score'})
            X, y = create_features_and_targets(event_data, target_column)

            # 检查数据是否足够训练模型
            if len(X) < 2:
                print(f"Insufficient data for {country} in {event}. Skipping...")
                continue

            # 分离训练特征和标签
            X_train = X[X['Year'] < 2028].drop(columns=['Year', 'NOC'])
            y_train = y[:len(X_train)]

            # 使用 GridSearchCV 进行超参数优化
            grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
            grid_search.fit(X_train, y_train)

            # 输出最佳参数
            # print(f"Best parameters for {country} in {event}: {grid_search.best_params_}")

            # 使用最佳参数训练模型
            best_model = grid_search.best_estimator_

            # 动态构建 2028 年特征
            feature_2028 = {}
            for col in X_train.columns:
                feature_2028[col] = X_train[col].mean()

            feature_2028['Is_Host'] = 1 if country == 'USA' else 0
            feature_2028 = pd.DataFrame([feature_2028])

            # 预测 2028 年的表现
            y_2028_pred = best_model.predict(feature_2028)

            # 构建预测结果
            all_results.append({
                'NOC': country,
                'Event': event,
                'Year': 2028,
                'Predicted_Medals': max(0, int(np.floor(y_2028_pred[0])))  # 保证非负整数
            })

    return pd.DataFrame(all_results)

# 数据预处理
bronze_data = optimized_complete_years(bronze_data)
gold_data = optimized_complete_years(gold_data)
silver_data = optimized_complete_years(silver_data)

# 将填补的年份数据保存为 CSV 文件
bronze_data.to_csv('bronze_data.csv', index=False)
gold_data.to_csv('gold_data.csv', index=False)
silver_data.to_csv('silver_data.csv', index=False)

# 预测
gold_predictions = predict_2028_per_country(gold_data, 'Gold')
silver_predictions = predict_2028_per_country(silver_data, 'Silver')
bronze_predictions = predict_2028_per_country(bronze_data, 'Bronze')

# 保存预测结果为 Excel 文件
gold_predictions.to_excel('gold_predictions_2028.xlsx', index=False)
silver_predictions.to_excel('silver_predictions_2028.xlsx', index=False)
bronze_predictions.to_excel('bronze_predictions_2028.xlsx', index=False)


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits


E:\Anaconda\envs\data_process\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits


E:\Anaconda\envs\data_process\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 5 folds for each of 144 candidates, totalling 720 fits


...

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits


E:\Anaconda\envs\data_process\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 5 folds for each of 144 candidates, totalling 720 fits


E:\Anaconda\envs\data_process\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits
